In [1]:
# Disable Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.9 MB 193 kB/s 


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import os

In [4]:
# mediapose utils

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# Video Checker

In [5]:
def video_checker(video_path):
    # Establishing connection with the video
    cap = cv2.VideoCapture(video_path)

    # Properties
    # extracting video properties
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    size = (width, height)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc('a','v','c','1')
    op_path = os.path.join('data', 'output_videos', 'output.avi')
    # Video Writer 
    video_writer = cv2.VideoWriter(op_path, fourcc, fps, size)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()

            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
            
            video_writer.write(image)

        # Releasing the video capture device
        cap.release()
        # Releasing video writer
        video_writer.release()

In [ ]:
video_checker(vid3_path)

NameError: ignored

# Video Checker with white background

In [ ]:
# setting background path
bg_path = os.path.join('data', 'white_bg.jpg')
bg_path

'data/white_bg.jpg'

In [ ]:
bg_path = os.path.join('data', 'white_bg.jpg')
bg_main = cv2.imread(bg_path)
bg_height, bg_width = bg_main.shape[0], bg_main.shape[1]

def video_checker_bg(video_path):
    # Establishing connection with the video
    cap = cv2.VideoCapture(video_path)

    # Properties
    # extracting video properties
    height = bg_height
    width = bg_width
    size = (width, height)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc('a','v','c','1')
    op_path = os.path.join('data', 'output_videos', 'output.avi')
    # Video Writer
    video_writer = cv2.VideoWriter(op_path, fourcc, fps, size)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()

            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # read the white background
            bg = cv2.imread(bg_path)
            
            # Render detections
            mp_drawing.draw_landmarks(bg, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
            
            video_writer.write(bg)

        # Releasing the video capture device
        cap.release()
        # Releasing video writer
        video_writer.release()

AttributeError: ignored

In [ ]:
video_checker_bg(vid1_path)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Ignoring empty camera frame.


# Visualization of the Box of rectangle per frame

In [ ]:
def video_checker2(video_path):
    # Establishing connection with the video
    cap = cv2.VideoCapture(video_path)

    # Properties
    # extracting video properties
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    size = (width, height)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc('a','v','c','1')
    op_path = os.path.join('data', 'output_videos', 'output.avi')
    # Video Writer 
    video_writer = cv2.VideoWriter(op_path, fourcc, fps, size)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()

            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass
            
            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z


            # Draw a rectangle in x-y plane, enclosing all the keypoints
            top_left = (x_min, y_max)
            bottom_right = (x_max, y_min)

            top_left = tuple(np.multiply(top_left, size).astype(int))
            bottom_right = tuple(np.multiply(bottom_right, size).astype(int))

            cv2.rectangle(image, top_left, bottom_right,(0,255,0),3)
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
            
            video_writer.write(image)

        # Releasing the video capture device
        cap.release()
        # Releasing video writer
        video_writer.release()

In [ ]:
video_checker2(vid3_path)

Ignoring empty camera frame.


# Visualization of the Box of rectangle on bg after rotation

In [ ]:
# this causes anti-clockwise rotation w.r.t. the +ve y axis by an angle of theta
def Ry(theta):
  return np.matrix([[ np.cos(theta), 0,  np.sin(theta)],
                    [ 0            , 1,  0            ],
                    [-np.sin(theta), 0,  np.cos(theta)]])

In [ ]:
X = np.matrix([
    [ 0, 0, 1],
    [ 0, 1, 0],
    [-1, 0, 0]
])
X.shape

(3, 3)

In [ ]:
y = np.matrix([
    [1],
    [1],
    [0]
])
y.shape

(3, 1)

In [ ]:
# (1,1,0) should go to (0,1,-1) after a rotation on 90 degrees anti-clockwise
X@y

matrix([[ 0],
        [ 1],
        [-1]])

In [ ]:
bg_path = os.path.join('data', 'white_bg.jpg')
bg_main = cv2.imread(bg_path)
bg_height, bg_width = bg_main.shape[0], bg_main.shape[1]

def video_checker_rot_bg(video_path, rot_angle = np.pi/4):
    # Establishing connection with the video
    cap = cv2.VideoCapture(video_path)

    # Properties
    # extracting video properties
    height = bg_height
    width = bg_width
    size = (width, height)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc('a','v','c','1')
    op_path = os.path.join('data', 'output_videos', 'output.avi')
    # Video Writer
    video_writer = cv2.VideoWriter(op_path, fourcc, fps, size)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()

            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            frame_coordinates = np.zeros((33,3))
            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                frame_coordinates[i][0],frame_coordinates[i][1],frame_coordinates[i][2] = landmarks[i].x, landmarks[i].y, landmarks[i].z
            
            # rotation matrix
            rot_mat_y = Ry(rot_angle)
            
            print(frame_coordinates.shape)

            # updated frame coordinates after rotating this w.r.t. y-axis
            frame_coordinates = frame_coordinates @ np.transpose(rot_mat_y)

            print(frame_coordinates.shape)

            # update (x,y,z) coordinates for each landmark in results
            for i in range(33):
                #landmarks[i].x, landmarks[i].y, landmarks[i].z = frame_coordinates[i][0],frame_coordinates[i][1],frame_coordinates[i][2]
                landmarks[i].x = frame_coordinates[i][0]
        
            # read the white background
            bg = cv2.imread(bg_path)
            
            # Render detections
            mp_drawing.draw_landmarks(bg, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
            
            video_writer.write(bg)

        # Releasing the video capture device
        cap.release()
        # Releasing video writer
        video_writer.release()

In [ ]:
def video_checker2(video_path):
    # Establishing connection with the video
    cap = cv2.VideoCapture(video_path)

    # Properties
    # extracting video properties
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    size = (width, height)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc('a','v','c','1')
    op_path = os.path.join('data', 'output_videos', 'output.avi')
    # Video Writer 
    video_writer = cv2.VideoWriter(op_path, fourcc, fps, size)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()

            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass
            
            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            # initializing overall x,z, min-max coordinates for finding axis of rotation
            X_min, Z_min = np.inf, np.inf
            X_max, Z_max = -np.inf, -np.inf

            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z
            
            # checking and updating overall min coordinates
            if x_min < X_min:
                X_min = x_min
            if z_min < Z_min:
                Z_min = z_min
            
            # checking and updating overall max coordinates
            if x_max > X_max:
                X_max = x_max
            if z_max > Z_max:
                Z_max = z_max


            # Draw a rectangle in x-y plane, enclosing all the keypoints
            top_left = (x_min, y_max)
            bottom_right = (x_max, y_min)

            top_left = tuple(np.multiply(top_left, size).astype(int))
            bottom_right = tuple(np.multiply(bottom_right, size).astype(int))

            cv2.rectangle(image, top_left, bottom_right,(0,255,0),3)
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
            
            video_writer.write(image)

        # Releasing the video capture device
        cap.release()
        # Releasing video writer
        video_writer.release()

# Video Paths

In [ ]:
vid1_path = os.path.join('data', 'sample_videos', 'curls3-1.mp4')
vid1_path

'data/sample_videos/curls3-1.mp4'

In [ ]:
vid2_path = os.path.join('data', 'sample_videos', 'curls3-2.mp4')
vid2_path

'data/sample_videos/curls3-2.mp4'

In [ ]:
vid3_path = os.path.join('data', 'sample_videos', 'curls3-3.mp4')
vid3_path

'data/sample_videos/curls3-3.mp4'

# VERSION 1.1 TEST

In [ ]:
## to create a helper function to extract (n, 33, 3) dimensional array to store the coordinates of each landmark in each frame

def extract_arr(input_video_path):
    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            frame_coordinates = np.zeros((33,3))
            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                frame_coordinates[i][0],frame_coordinates[i][1],frame_coordinates[i][2] = landmarks[i].x, landmarks[i].y, landmarks[i].z
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates)

In [ ]:
# reshape the (n, 33, 3) dimensional array to a (33, n, 3) dimensional array for extracting scores of each key-point

def my_reshape(input_arr):
    n_frames = input_arr.shape[0]
    n_landmarks = input_arr.shape[1]

    # new array of desired shape
    new_arr = np.zeros((n_landmarks, n_frames, 3))

    for f in range(n_frames):
        for i in range(n_landmarks):
            new_arr[i][f] = input_arr[f][i]
    
    return new_arr

In [ ]:
from dtaidistance.dtw_ndim import distance, distance_fast

## to compare 2 videos of dim (33, n, 3) using DTW (using strategy 2)

def compare_vid(vid1_arr, vid2_arr):
    n_landmarks = vid1_arr.shape[0]
    
    scores = np.zeros(n_landmarks)
    
    for i in range (n_landmarks):
        # normalize the coordinates
        vid1_arr[i] = vid1_arr[i]/np.linalg.norm(vid1_arr[i])
        vid2_arr[i] = vid2_arr[i]/np.linalg.norm(vid2_arr[i])
        
        # calculate distance
        d = distance(vid1_arr[i], vid2_arr[i])
        # we give a score out of 100
        d_score = 100 - (d*100)
        scores[i] = d_score
    
    return scores

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path):
    vid1_arr = extract_arr(vid1_path)
    vid2_arr  = extract_arr(vid2_path)

    vid1_arr_new = my_reshape(vid1_arr)
    vid2_arr_new = my_reshape(vid2_arr)

    scores = compare_vid(vid1_arr_new, vid2_arr_new)

    print("Scores List:\n", scores)
    print("Average Score:", scores.mean())

In [ ]:
%%time
combined_compare(vid1_path, vid1_path)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100.]
Average Score: 100.0
CPU times: user 21.4 s, sys: 403 ms, total: 21.8 s
Wall time: 21.1 s


In [ ]:
%%time
combined_compare(vid1_path, vid2_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [77.64975877 77.51754387 77.55602626 77.60267612 77.69823648 77.72383293
 77.76272665 77.87427589 78.52889323 77.89025921 78.09730396 79.23676164
 79.29086474 84.07772858 80.88352592 76.7209857  69.06288637 75.14808023
 67.02120509 74.80446291 66.30122747 74.34038988 67.36503401 80.99545498
 81.17073534 79.05177224 77.54065719 77.68160555 76.73332758 77.80852284
 76.83242531 76.98610586 76.16014382]
Average Score: 76.7004677761122
CPU times: user 16.8 s, sys: 319 ms, total: 17.1 s
Wall time: 16.3 s


In [ ]:
%%time
combined_compare(vid1_path, vid3_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [90.36712509 90.48350076 90.4214465  90.35077107 89.98707817 89.96469313
 89.93794297 90.28094662 88.2643645  90.54121127 89.79774844 91.68475828
 87.10831677 92.99999853 87.92232255 88.48792062 81.77806401 88.23772274
 80.51475224 88.83917342 81.23785002 87.93834118 81.52105547 92.34317024
 91.37831678 91.96892653 92.49469029 92.16970594 92.91024929 92.23795573
 92.89406741 92.15368687 92.04274271]
Average Score: 89.4321398829224
CPU times: user 20.1 s, sys: 354 ms, total: 20.5 s
Wall time: 19.7 s


# VERSION 1.2 TEST

In [ ]:
## to create a helper function to extract (n, len(landmark_list), 3) dimensional array to store the coordinates of each landmark in the landmark_list in each frame (we only consider the landmarks in this list `landmark_list`)

def extract_arr(input_video_path, landmark_list = list(range(33))):
    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            # we sort the landmark_list inplace for convenience
            landmark_list.sort()

            frame_coordinates = np.zeros((len(landmark_list),3))
            # extract (x,y,z) coordinate of each landmark
            for i in range(len(landmark_list)):
                idx = landmark_list[i]
                frame_coordinates[i][0],frame_coordinates[i][1],frame_coordinates[i][2] = landmarks[idx].x, landmarks[idx].y, landmarks[idx].z
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates), landmark_list

In [ ]:
# reshape the (n, 33, 3) dimensional array to a (33, n, 3) dimensional array for extracting scores of each key-point

def my_reshape(input_arr):
    n_frames = input_arr.shape[0]
    n_landmarks = input_arr.shape[1]

    # new array of desired shape
    new_arr = np.zeros((n_landmarks, n_frames, 3))

    for f in range(n_frames):
        for i in range(n_landmarks):
            new_arr[i][f] = input_arr[f][i]
    
    return new_arr

In [ ]:
from dtaidistance.dtw_ndim import distance, distance_fast

## to compare 2 videos of dim (33, n, 3) using DTW (using strategy 2)

def compare_vid(vid1_arr, vid2_arr):
    n_landmarks = vid1_arr.shape[0]
    
    scores = np.zeros(n_landmarks)
    
    for i in range (n_landmarks):
        # normalize the coordinates
        vid1_arr[i] = vid1_arr[i]/np.linalg.norm(vid1_arr[i])
        vid2_arr[i] = vid2_arr[i]/np.linalg.norm(vid2_arr[i])
        
        # calculate distance
        d = distance(vid1_arr[i], vid2_arr[i])
        # we give a score out of 100
        d_score = 100 - (d*100)
        scores[i] = d_score
    
    return scores

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path, landmark_list = list(range(33))):
    vid1_arr, _ = extract_arr(vid1_path, landmark_list)
    vid2_arr, _  = extract_arr(vid2_path, landmark_list)

    vid1_arr_new = my_reshape(vid1_arr)
    vid2_arr_new = my_reshape(vid2_arr)

    scores = compare_vid(vid1_arr_new, vid2_arr_new)

    print("Scores List:\n", scores)
    print("Average Score:", scores.mean())

In [ ]:
# we consider landmarks from 0 till 24
landmark_list = list(range(25))

In [ ]:
combined_compare(vid1_path, vid1_path, landmark_list)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
Average Score: 100.0


In [ ]:
combined_compare(vid1_path, vid2_path, landmark_list)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [77.64975877 77.51754387 77.55602626 77.60267612 77.69823648 77.72383293
 77.76272665 77.87427589 78.52889323 77.89025921 78.09730396 79.23676164
 79.29086474 84.07772858 80.88352592 76.7209857  69.06288637 75.14808023
 67.02120509 74.80446291 66.30122747 74.34038988 67.36503401 80.99545498
 81.17073534]
Average Score: 76.49283504900352


In [ ]:
combined_compare(vid1_path, vid3_path, landmark_list)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [90.36712509 90.48350076 90.4214465  90.35077107 89.98707817 89.96469313
 89.93794297 90.28094662 88.2643645  90.54121127 89.79774844 91.68475828
 87.10831677 92.99999853 87.92232255 88.48792062 81.77806401 88.23772274
 80.51475224 88.83917342 81.23785002 87.93834118 81.52105547 92.34317024
 91.37831678]
Average Score: 88.4955436546632


# VERSION 2.1 TEST

In [ ]:
## to create a helper function to extract (n, 33, 3) dimensional array to store the coordinates of each landmark in each frame

def extract_arr(input_video_path):
    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            frame_coordinates = np.zeros((33,3))
            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                frame_coordinates[i][0],frame_coordinates[i][1],frame_coordinates[i][2] = landmarks[i].x, landmarks[i].y, landmarks[i].z
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates)

In [ ]:
from dtaidistance.dtw_ndim import distance, distance_fast

def compare_vid(vid1_arr, vid2_arr):
    n_landmarks = vid1_arr.shape[0]
    
    # normalize the coordinates
    vid1_arr = vid1_arr/np.linalg.norm(vid1_arr)
    vid2_arr = vid2_arr/np.linalg.norm(vid2_arr)
    
    d = distance(vid1_arr, vid2_arr)
    d_score = 100 - (d*100)
    
    return d_score

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path):
    vid1_arr = extract_arr(vid1_path)
    vid2_arr  = extract_arr(vid2_path)

    score = compare_vid(vid1_arr, vid2_arr)

    print("Final Score:", score)

In [ ]:
combined_compare(vid1_path, vid1_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Final Score: 100.0


In [ ]:
combined_compare(vid1_path, vid2_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Final Score: 72.13579475938042


In [ ]:
combined_compare(vid1_path, vid3_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Final Score: 85.61251023753023


# VERSION 2.2 TEST

In [ ]:
## to create a helper function to extract (n, len(landmark_list), 3) dimensional array to store the coordinates of each landmark in the landmark_list in each frame (we only consider the landmarks in this list `landmark_list`)

def extract_arr(input_video_path, landmark_list = list(range(33))):
    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            # we sort the landmark_list inplace for convenience
            landmark_list.sort()

            frame_coordinates = np.zeros((len(landmark_list),3))
            # extract (x,y,z) coordinate of each landmark
            for i in range(len(landmark_list)):
                idx = landmark_list[i]
                frame_coordinates[i][0],frame_coordinates[i][1],frame_coordinates[i][2] = landmarks[idx].x, landmarks[idx].y, landmarks[idx].z
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates), landmark_list

In [ ]:
from dtaidistance.dtw_ndim import distance, distance_fast

def compare_vid(vid1_arr, vid2_arr):
    
    # normalize the coordinates
    vid1_arr = vid1_arr/np.linalg.norm(vid1_arr)
    vid2_arr = vid2_arr/np.linalg.norm(vid2_arr)
    
    d = distance(vid1_arr, vid2_arr)
    d_score = 100 - (d*100)
    
    return d_score

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path, landmark_list = list(range(33))):
    vid1_arr, _ = extract_arr(vid1_path, landmark_list)
    vid2_arr, _ = extract_arr(vid2_path, landmark_list)

    score = compare_vid(vid1_arr, vid2_arr)

    print("Final Score:", score)

In [ ]:
# we consider landmarks from 0 till 24
landmark_list = list(range(25))

In [ ]:
combined_compare(vid1_path, vid1_path,landmark_list)

Ignoring empty camera frame.
Ignoring empty camera frame.
Final Score: 100.0


In [ ]:
combined_compare(vid1_path, vid2_path, landmark_list)

Ignoring empty camera frame.
Ignoring empty camera frame.
Final Score: 70.6449503249911


In [ ]:
combined_compare(vid1_path, vid3_path, landmark_list)

Ignoring empty camera frame.
Ignoring empty camera frame.
Final Score: 83.77952859907592


# VERSION 3.1 TEST: Normalizing the subject

## Idea: Capturing Subject in a cuboid
- Capture min-max for x, y and z coordinates to extract a cuboid
- Normalize into a unit lengthed cube
- Translation of  min coordinate to 0
- Repeat the above steps in each dimension

In [ ]:
## to create a helper function to extract (n, 33, 3) dimensional array to store the coordinates of each landmark in each frame

## min_coordinates, max_coordinates = (n, 3), (n, 3)

def extract_arr(input_video_path):
    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            # initialize to store the key-points coordinates in the frame
            frame_coordinates = np.zeros((33,3))
            
            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z
                frame_coordinates[i][0], frame_coordinates[i][1], frame_coordinates[i][2] = x, y, z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z   
            
            # normalize frame_coordinates
            for i in range(33):
                # normalizing x coordinate
                frame_coordinates[i][0] = (frame_coordinates[i][0] - x_min)/(x_max - x_min)
                # normalizing y coordinate
                frame_coordinates[i][1] = (frame_coordinates[i][1] - y_min)/(y_max - y_min)
                # normalizing z coordinate
                frame_coordinates[i][2] = (frame_coordinates[i][2] - z_min)/(z_max - z_min)
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates)

In [ ]:
# reshape the (n, 33, 3) dimensional array to a (33, n, 3) dimensional array for extracting scores of each key-point

def my_reshape(input_arr):
    n_frames = input_arr.shape[0]
    n_landmarks = input_arr.shape[1]

    # new array of desired shape
    new_arr = np.zeros((n_landmarks, n_frames, 3))

    for f in range(n_frames):
        for i in range(n_landmarks):
            new_arr[i][f] = input_arr[f][i]
    
    return new_arr

In [ ]:
from dtaidistance.dtw_ndim import distance, distance_fast

## to compare 2 videos of dim (33, n, 3) using DTW (using strategy 2)

def compare_vid(vid1_arr, vid2_arr):
    n_landmarks = vid1_arr.shape[0]
    
    scores = np.zeros(n_landmarks)
    
    for i in range (n_landmarks):
        # normalize the coordinates
        vid1_arr[i] = vid1_arr[i]/np.linalg.norm(vid1_arr[i])
        vid2_arr[i] = vid2_arr[i]/np.linalg.norm(vid2_arr[i])
        
        # calculate distance
        d = distance(vid1_arr[i], vid2_arr[i])
        # we give a score out of 100
        d_score = 100 - (d*100)
        scores[i] = d_score
    
    return scores

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path):
    vid1_arr = extract_arr(vid1_path)
    vid2_arr  = extract_arr(vid2_path)

    vid1_arr_new = my_reshape(vid1_arr)
    vid2_arr_new = my_reshape(vid2_arr)

    scores = compare_vid(vid1_arr_new, vid2_arr_new)

    print("Scores List:\n", scores)
    print("Average Score:", scores.mean())

In [ ]:
%%time
combined_compare(vid1_path, vid1_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100.]
Average Score: 100.0
CPU times: user 34.8 s, sys: 643 ms, total: 35.5 s
Wall time: 34.1 s


In [ ]:
%%time
combined_compare(vid1_path, vid2_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [70.64647839 72.04212754 72.4391972  72.87225625 70.89557378 71.13176141
 71.47964269 76.64926065 73.51485129 73.78181789 72.46286718 81.5419961
 79.96752798 66.27161632 75.31776237 59.18279573 65.48069597 52.85263092
 59.70680572 52.3766379  57.8763038  55.54283735 61.07838155 74.01273037
 75.66878705 73.65711004 77.58279603 79.34605448 79.69506302 80.02573554
 80.03066776 78.59057671 80.22067364]
Average Score: 71.02854607963663
CPU times: user 27.2 s, sys: 514 ms, total: 27.7 s
Wall time: 26.5 s


In [ ]:
%%time
combined_compare(vid1_path, vid3_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [76.09880093 71.03308007 71.92827159 72.97813549 71.17779816 71.32982945
 71.62863353 83.42066047 80.79698083 83.79111416 83.92837262 91.99387374
 89.42282927 89.75248382 84.20653405 80.34372698 74.62355477 77.20345952
 71.53413893 77.28875337 72.1713712  78.79991038 74.87175014 87.50617816
 86.33696332 82.74433482 88.15196298 92.0240622  93.22718093 93.21799267
 93.55924829 90.54064646 93.09000439]
Average Score: 81.84007992994708
CPU times: user 32.7 s, sys: 622 ms, total: 33.3 s
Wall time: 32 s


# VERSION 3.2 TEST

In [ ]:
## to create a helper function to extract (n, len(landmark_list), 3) dimensional array to store the coordinates of each landmark in the landmark_list in each frame (we only consider the landmarks in this list `landmark_list`)

def extract_arr(input_video_path, landmark_list = list(range(33))):
    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            # we sort the landmark_list inplace for convenience
            landmark_list.sort()
            
            # initialize to store the key-points coordinates in the frame
            frame_coordinates = np.zeros((len(landmark_list),3))
            
            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            # extract (x,y,z) coordinate of each landmark
            for i in range(len(landmark_list)):
                idx = landmark_list[i]
                x, y, z = landmarks[idx].x, landmarks[idx].y, landmarks[idx].z
                frame_coordinates[i][0], frame_coordinates[i][1], frame_coordinates[i][2] = x, y, z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z   
            
            # normalize frame_coordinates
            for i in range(len(landmark_list)):
                # normalizing x coordinate
                frame_coordinates[i][0] = (frame_coordinates[i][0] - x_min)/(x_max - x_min)
                # normalizing y coordinate
                frame_coordinates[i][1] = (frame_coordinates[i][1] - y_min)/(y_max - y_min)
                # normalizing z coordinate
                frame_coordinates[i][2] = (frame_coordinates[i][2] - z_min)/(z_max - z_min)
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates), landmark_list

In [ ]:
# reshape the (n, len(landmark_list), 3) dimensional array to a (len(landmark_list), n, 3) dimensional array for extracting scores of each key-point

def my_reshape(input_arr):
    n_frames = input_arr.shape[0]
    n_landmarks = input_arr.shape[1]

    # new array of desired shape
    new_arr = np.zeros((n_landmarks, n_frames, 3))

    for f in range(n_frames):
        for i in range(n_landmarks):
            new_arr[i][f] = input_arr[f][i]
    
    return new_arr

In [ ]:
from dtaidistance.dtw_ndim import distance, distance_fast

## to compare 2 videos of dim (33, n, 3) using DTW (using strategy 2)

def compare_vid(vid1_arr, vid2_arr):
    n_landmarks = vid1_arr.shape[0]
    
    scores = np.zeros(n_landmarks)
    
    for i in range (n_landmarks):
        # normalize the coordinates
        vid1_arr[i] = vid1_arr[i]/np.linalg.norm(vid1_arr[i])
        vid2_arr[i] = vid2_arr[i]/np.linalg.norm(vid2_arr[i])
        
        # calculate distance
        d = distance(vid1_arr[i], vid2_arr[i])
        # we give a score out of 100
        d_score = 100 - (d*100)
        scores[i] = d_score
    
    return scores

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path, landmark_list = list(range(33))):
    vid1_arr, _ = extract_arr(vid1_path, landmark_list)
    vid2_arr, _  = extract_arr(vid2_path, landmark_list)

    vid1_arr_new = my_reshape(vid1_arr)
    vid2_arr_new = my_reshape(vid2_arr)

    scores = compare_vid(vid1_arr_new, vid2_arr_new)

    print("Scores List:\n", scores)
    print("Average Score:", scores.mean())

In [ ]:
# we consider landmarks from 0 till 24
landmark_list = list(range(25))

In [ ]:
%%time
combined_compare(vid1_path, vid1_path, landmark_list)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
Average Score: 100.0
CPU times: user 19.4 s, sys: 462 ms, total: 19.9 s
Wall time: 19.1 s


In [ ]:
%%time
combined_compare(vid1_path, vid2_path, landmark_list)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [77.35746198 77.4755386  77.96513494 78.49942307 77.01953939 77.18586414
 77.48336462 81.13451253 79.70358058 79.9590351  79.50166629 81.43148999
 85.53170833 65.59291866 76.88160367 52.32549177 69.21050326 49.27063941
 66.29886869 49.27494289 64.55502344 50.97623    66.85073321 75.39640356
 76.66159941]
Average Score: 71.7417311011076
CPU times: user 15.2 s, sys: 332 ms, total: 15.6 s
Wall time: 14.9 s


In [ ]:
%%time
combined_compare(vid1_path, vid3_path, landmark_list)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [79.29076533 74.62334593 75.25919102 76.04498242 76.75394097 76.75762224
 76.875806   85.18942254 85.8666015  85.01640652 86.24877752 93.15801846
 88.64307079 87.08987375 83.57118547 88.67744092 74.7101584  87.44175462
 73.97601561 87.14164671 75.5954874  87.54069156 76.52657353 84.99043884
 85.98794896]
Average Score: 82.11908668037474
CPU times: user 18 s, sys: 343 ms, total: 18.4 s
Wall time: 17.6 s


# VERSION 4.1 TEST

In [ ]:
## to create a helper function to extract (n, 33, 3) dimensional array to store the coordinates of each landmark in each frame

def extract_arr(input_video_path):
    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            # initialize to store the key-points coordinates in the frame
            frame_coordinates = np.zeros((33,3))
            
            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z
                frame_coordinates[i][0], frame_coordinates[i][1], frame_coordinates[i][2] = x, y, z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z

            # normalize frame_coordinates
            for i in range(33):
                # normalizing x coordinate
                frame_coordinates[i][0] = (frame_coordinates[i][0] - x_min)/(x_max - x_min)
                # normalizing y coordinate
                frame_coordinates[i][1] = (frame_coordinates[i][1] - y_min)/(y_max - y_min)
                # normalizing z coordinate
                frame_coordinates[i][2] = (frame_coordinates[i][2] - z_min)/(z_max - z_min)
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates)

In [ ]:
from dtaidistance.dtw_ndim import distance, distance_fast

def compare_vid(vid1_arr, vid2_arr):
    n_landmarks = vid1_arr.shape[0]
    
    # normalize the coordinates
    vid1_arr = vid1_arr/np.linalg.norm(vid1_arr)
    vid2_arr = vid2_arr/np.linalg.norm(vid2_arr)
    
    d = distance(vid1_arr, vid2_arr)
    d_score = 100 - (d*100)
    
    return d_score

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path):
    vid1_arr = extract_arr(vid1_path)
    vid2_arr  = extract_arr(vid2_path)

    score = compare_vid(vid1_arr, vid2_arr)

    print("Final Score:", score)

In [ ]:
%%time
combined_compare(vid1_path, vid1_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Final Score: 100.0
CPU times: user 12 s, sys: 336 ms, total: 12.3 s
Wall time: 11.4 s


In [ ]:
%%time
combined_compare(vid1_path, vid2_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Final Score: 67.32642349397406
CPU times: user 10.2 s, sys: 365 ms, total: 10.6 s
Wall time: 9.9 s


In [ ]:
%%time
combined_compare(vid1_path, vid3_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Final Score: 80.65988501532588
CPU times: user 11.4 s, sys: 357 ms, total: 11.8 s
Wall time: 10.9 s


# VERSION 5.1: Including Sensitivity on top of Normalization

In [ ]:
## to create a helper function to extract (n, 33, 3) dimensional array to store the coordinates of each landmark in each frame

## min_coordinates, max_coordinates = (n, 3), (n, 3)

def extract_arr(input_video_path):
    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            # initialize to store the key-points coordinates in the frame
            frame_coordinates = np.zeros((33,3))
            
            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z
                frame_coordinates[i][0], frame_coordinates[i][1], frame_coordinates[i][2] = x, y, z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z   
            
            # normalize frame_coordinates
            for i in range(33):
                # normalizing x coordinate
                frame_coordinates[i][0] = (frame_coordinates[i][0] - x_min)/(x_max - x_min)
                # normalizing y coordinate
                frame_coordinates[i][1] = (frame_coordinates[i][1] - y_min)/(y_max - y_min)
                # normalizing z coordinate
                frame_coordinates[i][2] = (frame_coordinates[i][2] - z_min)/(z_max - z_min)
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates)

In [ ]:
# reshape the (n, 33, 3) dimensional array to a (33, n, 3) dimensional array for extracting scores of each key-point

def my_reshape(input_arr):
    n_frames = input_arr.shape[0]
    n_landmarks = input_arr.shape[1]

    # new array of desired shape
    new_arr = np.zeros((n_landmarks, n_frames, 3))

    for f in range(n_frames):
        for i in range(n_landmarks):
            new_arr[i][f] = input_arr[f][i]
    
    return new_arr

In [ ]:
from dtaidistance.dtw_ndim import distance, distance_fast

## to compare 2 videos of dim (33, n, 3) using DTW (using strategy 2)

def compare_vid(vid1_arr, vid2_arr, senstivity = 1):
    n_landmarks = vid1_arr.shape[0]
    
    scores = np.zeros(n_landmarks)
    
    for i in range (n_landmarks):
        # normalize the coordinates using sensitivity; higher sensitivity => more lenient scoring, lower sensitivity => strict scoring
        vid1_arr[i] = vid1_arr[i]/(np.linalg.norm(vid1_arr[i]) * senstivity)
        vid2_arr[i] = vid2_arr[i]/(np.linalg.norm(vid2_arr[i]) * senstivity)
        
        # calculate distance
        d = distance(vid1_arr[i], vid2_arr[i])
        # we give a score out of 100
        d_score = 100 - (d*100)
        scores[i] = d_score
    
    return scores

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path, senstivity=1):
    vid1_arr = extract_arr(vid1_path)
    vid2_arr  = extract_arr(vid2_path)

    vid1_arr_new = my_reshape(vid1_arr)
    vid2_arr_new = my_reshape(vid2_arr)

    scores = compare_vid(vid1_arr_new, vid2_arr_new, senstivity)

    print("Scores List:\n", scores)
    print("Average Score:", scores.mean())

### `Sensitivity = 0.5`

In [ ]:
%%time
combined_compare(vid1_path, vid1_path, 0.5)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100.]
Average Score: 100.0
CPU times: user 35 s, sys: 708 ms, total: 35.8 s
Wall time: 34.4 s


In [ ]:
%%time
combined_compare(vid1_path, vid2_path, 0.5)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [41.29295678 44.08425509 44.87839439 45.74451249 41.79114755 42.26352283
 42.95928539 53.2985213  47.02970258 47.56363577 44.92573436 63.08399221
 59.93505596 32.54323265 50.63552474 18.36559147 30.96139195  5.70526184
 19.41361143  4.7532758  15.7526076  11.08567471 22.1567631  48.02546074
 51.3375741  47.31422008 55.16559206 58.69210896 59.39012603 60.05147107
 60.06133551 57.18115343 60.44134728]
Average Score: 42.05709215927326
CPU times: user 27 s, sys: 522 ms, total: 27.5 s
Wall time: 26.4 s


In [ ]:
%%time
combined_compare(vid1_path, vid3_path, 0.5)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [52.19760186 42.06616013 43.85654318 45.95627098 42.35559631 42.65965891
 43.25726706 66.84132094 61.59396165 67.58222832 67.85674525 83.98774748
 78.84565854 79.50496764 68.4130681  60.68745396 49.24710954 54.40691904
 43.06827785 54.57750674 44.3427424  57.59982076 49.74350029 75.01235632
 72.67392663 65.48866964 76.30392597 84.0481244  86.45436187 86.43598534
 87.11849659 81.08129293 86.18000878]
Average Score: 63.68015985989418
CPU times: user 32.3 s, sys: 609 ms, total: 32.9 s
Wall time: 31.7 s


### `Sensitivity = 10`

In [ ]:
%%time
combined_compare(vid1_path, vid1_path, 10)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100.]
Average Score: 100.0
CPU times: user 21.8 s, sys: 444 ms, total: 22.3 s
Wall time: 21.5 s


In [ ]:
%%time
combined_compare(vid1_path, vid2_path, 10)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [97.06464784 97.20421275 97.24391972 97.28722562 97.08955738 97.11317614
 97.14796427 97.66492606 97.35148513 97.37818179 97.24628672 98.15419961
 97.9967528  96.62716163 97.53177624 95.91827957 96.5480696  95.28526309
 95.97068057 95.23766379 95.78763038 95.55428374 96.10783816 97.40127304
 97.56687871 97.365711   97.7582796  97.93460545 97.9695063  98.00257355
 98.00306678 97.85905767 98.02206736]
Average Score: 97.10285460796366
CPU times: user 17 s, sys: 319 ms, total: 17.3 s
Wall time: 16.5 s


In [ ]:
%%time
combined_compare(vid1_path, vid3_path, 10)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [97.60988009 97.10330801 97.19282716 97.29781355 97.11777982 97.13298295
 97.16286335 98.34206605 98.07969808 98.37911142 98.39283726 99.19938737
 98.94228293 98.97524838 98.42065341 98.0343727  97.46235548 97.72034595
 97.15341389 97.72887534 97.21713712 97.87999104 97.48717501 98.75061782
 98.63369633 98.27443348 98.8151963  99.20240622 99.32271809 99.32179927
 99.35592483 99.05406465 99.30900044]
Average Score: 98.18400799299471
CPU times: user 20.4 s, sys: 373 ms, total: 20.7 s
Wall time: 20.1 s


It seems `sensitivity=10` is too lenient.

### `Sensitivity = 5`

In [ ]:
%%time
combined_compare(vid1_path, vid1_path, 5)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100.]
Average Score: 100.0
CPU times: user 21.8 s, sys: 464 ms, total: 22.3 s
Wall time: 21.7 s


In [ ]:
%%time
combined_compare(vid1_path, vid2_path, 5)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [94.12929568 94.40842551 94.48783944 94.57445125 94.17911476 94.22635228
 94.29592854 95.32985213 94.70297026 94.75636358 94.49257344 96.30839922
 95.9935056  93.25432326 95.06355247 91.83655915 93.09613919 90.57052618
 91.94136114 90.47532758 91.57526076 91.10856747 92.21567631 94.80254607
 95.13375741 94.73142201 95.51655921 95.8692109  95.9390126  96.00514711
 96.00613355 95.71811534 96.04413473]
Average Score: 94.20570921592733
CPU times: user 17.2 s, sys: 372 ms, total: 17.5 s
Wall time: 16.8 s


In [ ]:
%%time
combined_compare(vid1_path, vid3_path, 5)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [95.21976019 94.20661601 94.38565432 94.5956271  94.23555963 94.26596589
 94.32572671 96.68413209 96.15939617 96.75822283 96.78567452 98.39877475
 97.88456585 97.95049676 96.84130681 96.0687454  94.92471095 95.4406919
 94.30682779 95.45775067 94.43427424 95.75998208 94.97435003 97.50123563
 97.26739266 96.54886696 97.6303926  98.40481244 98.64543619 98.64359853
 98.71184966 98.10812929 98.61800088]
Average Score: 96.36801598598942
CPU times: user 20.4 s, sys: 374 ms, total: 20.8 s
Wall time: 19.9 s


### `Sensitivity = 3`

In [ ]:
%%time
combined_compare(vid1_path, vid1_path, 3)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100.]
Average Score: 100.0
CPU times: user 22.2 s, sys: 472 ms, total: 22.7 s
Wall time: 21.8 s


In [ ]:
%%time
combined_compare(vid1_path, vid2_path, 3)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [90.2154928  90.68070918 90.81306573 90.95741875 90.29852459 90.3772538
 90.49321423 92.21642022 91.1716171  91.26060596 90.82095573 93.84733203
 93.32250933 88.75720544 91.77258746 86.39426524 88.49356532 84.28421031
 86.56893524 84.12554597 85.95876793 85.18094578 87.02612718 91.33757679
 91.88959568 91.21903668 92.52759868 93.11535149 93.23168767 93.34191185
 93.34355592 92.86352557 93.40689121]
Average Score: 90.34284869321222
CPU times: user 17.1 s, sys: 350 ms, total: 17.4 s
Wall time: 16.7 s


In [ ]:
%%time
combined_compare(vid1_path, vid3_path, 3)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [92.03293364 90.34436002 90.6427572  90.99271183 90.39259939 90.44327648
 90.54287784 94.47355349 93.59899361 94.59703805 94.64279087 97.33129125
 96.47427642 96.58416127 94.73551135 93.44790899 91.54118492 92.40115317
 90.51137964 92.42958446 90.7237904  92.93330346 91.62391671 95.83539272
 95.44565444 94.24811161 96.05065433 97.34135407 97.74239364 97.73933089
 97.85308276 96.84688215 97.69666813]
Average Score: 93.94669330998236
CPU times: user 20.1 s, sys: 398 ms, total: 20.5 s
Wall time: 19.8 s


### `Sensitivity = 1.5`

In [ ]:
%%time
combined_compare(vid1_path, vid1_path, 1.5)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100.]
Average Score: 100.0
CPU times: user 21.7 s, sys: 440 ms, total: 22.1 s
Wall time: 21.4 s


In [ ]:
%%time
combined_compare(vid1_path, vid2_path, 1.5)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [80.43098559 81.36141836 81.62613146 81.9148375  80.59704918 80.75450761
 80.98642846 84.43284043 82.34323419 82.52121192 81.64191145 87.69466407
 86.64501865 77.51441088 83.54517491 72.78853049 76.98713065 68.56842061
 73.13787048 68.25109193 71.91753587 70.36189157 74.05225437 82.67515358
 83.77919137 82.43807336 85.05519735 86.23070299 86.46337534 86.68382369
 86.68711184 85.72705114 86.81378243]
Average Score: 80.68569738642442
CPU times: user 17.1 s, sys: 358 ms, total: 17.5 s
Wall time: 16.7 s


In [ ]:
%%time
combined_compare(vid1_path, vid3_path, 1.5)

Ignoring empty camera frame.
Ignoring empty camera frame.
Scores List:
 [84.06586729 80.68872004 81.28551439 81.98542366 80.78519877 80.88655297
 81.08575569 88.94710698 87.19798722 89.19407611 89.28558175 94.66258249
 92.94855285 93.16832255 89.4710227  86.89581799 83.08236985 84.80230635
 81.02275928 84.85916891 81.4475808  85.86660692 83.24783343 91.67078544
 90.89130888 88.49622321 92.10130866 94.68270813 95.48478729 95.47866178
 95.70616553 93.69376431 95.39333626]
Average Score: 87.89338661996472
CPU times: user 20.3 s, sys: 323 ms, total: 20.6 s
Wall time: 19.8 s


It seems like `sensitivity = 1.5` seems like a sweet spot.

# Video Paths for newer testing (squats)

In [ ]:
# right videos
r1_path = os.path.join('data', 'sample_videos', 'R1.mp4')
r2_path = os.path.join('data', 'sample_videos', 'R2.mp4')
r3_path = os.path.join('data', 'sample_videos', 'R3.mp4')
r4_path = os.path.join('data', 'sample_videos', 'R4.mp4')

In [ ]:
# wrong videos
w1_path = os.path.join('data', 'sample_videos', 'W1.mp4')
w2_path = os.path.join('data', 'sample_videos', 'W2.mp4')
w3_path = os.path.join('data', 'sample_videos', 'W3.mp4')
w4_path = os.path.join('data', 'sample_videos', 'W4.mp4')

# Testing using VERSION 3.1

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path):
    vid1_arr = extract_arr(vid1_path)
    vid2_arr  = extract_arr(vid2_path)

    vid1_arr_new = my_reshape(vid1_arr)
    vid2_arr_new = my_reshape(vid2_arr)

    scores = compare_vid(vid1_arr_new, vid2_arr_new)

    return scores.mean()

In [ ]:
r_path = [r1_path, r2_path, r3_path, r4_path]
w_path = [w1_path, w2_path, w3_path, w4_path]

In [ ]:
r_score = 0
r_count = 0

for i in range(len(r_path)):
    for j in range(i, len(r_path)):
        r_score += combined_compare(r_path[i], r_path[j])
        r_count += 1

print("Average Right Video Comparison Score:", r_score/r_count)

Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Average Right Video Comparison Score: 80.13880997915103


In [ ]:
w_score = 0
w_count = 0

for i in range(len(r_path)):
    for j in range(len(w_path)):
        w_score += combined_compare(r_path[i], w_path[j])
        w_count += 1

print("Average Wrong Video Comparison Score:", w_score/w_count)

Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Average Wrong Video Comparison Score: 66.92426199589143


# VERSION 6.1 TEST

Now, we try comparing the current video with multiple trainer videos and consider the best score out of the lot.

In [ ]:
## to create a helper function to extract (n, 33, 3) dimensional array to store the coordinates of each landmark in each frame

## min_coordinates, max_coordinates = (n, 3), (n, 3)

def extract_arr(input_video_path):
    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            # initialize to store the key-points coordinates in the frame
            frame_coordinates = np.zeros((33,3))
            
            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z
                frame_coordinates[i][0], frame_coordinates[i][1], frame_coordinates[i][2] = x, y, z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z   
            
            # normalize frame_coordinates
            for i in range(33):
                # normalizing x coordinate
                frame_coordinates[i][0] = (frame_coordinates[i][0] - x_min)/(x_max - x_min)
                # normalizing y coordinate
                frame_coordinates[i][1] = (frame_coordinates[i][1] - y_min)/(y_max - y_min)
                # normalizing z coordinate
                frame_coordinates[i][2] = (frame_coordinates[i][2] - z_min)/(z_max - z_min)
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates)

In [ ]:
# reshape the (n, 33, 3) dimensional array to a (33, n, 3) dimensional array for extracting scores of each key-point

def my_reshape(input_arr):
    n_frames = input_arr.shape[0]
    n_landmarks = input_arr.shape[1]

    # new array of desired shape
    new_arr = np.zeros((n_landmarks, n_frames, 3))

    for f in range(n_frames):
        for i in range(n_landmarks):
            new_arr[i][f] = input_arr[f][i]
    
    return new_arr

In [ ]:
from dtaidistance.dtw_ndim import distance, distance_fast

## to compare 2 videos of dim (33, n, 3) using DTW (using strategy 2)

def compare_vid(vid1_arr, vid2_arr):
    n_landmarks = vid1_arr.shape[0]
    
    scores = np.zeros(n_landmarks)
    
    for i in range (n_landmarks):
        # normalize the coordinates
        vid1_arr[i] = vid1_arr[i]/np.linalg.norm(vid1_arr[i])
        vid2_arr[i] = vid2_arr[i]/np.linalg.norm(vid2_arr[i])
        
        # calculate distance
        d = distance(vid1_arr[i], vid2_arr[i])
        # we give a score out of 100
        d_score = 100 - (d*100)
        scores[i] = d_score
    
    return scores.mean()

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid_path):
    # input videos
    vid_arr = extract_arr(vid_path)
    vid_arr_new = my_reshape(vid_arr)

    # trainer videos
    r1_arr = extract_arr(r1_path)
    r1_arr_new = my_reshape(r1_arr)
    r3_arr = extract_arr(r3_path)
    r3_arr_new = my_reshape(r3_arr)
    r4_arr = extract_arr(r4_path)
    r4_arr_new = my_reshape(r4_arr)

    score = max(compare_vid(vid_arr_new, r1_arr_new),
                compare_vid(vid_arr_new, r3_arr_new),
                compare_vid(vid_arr_new, r4_arr_new))

    return score

In [ ]:
%%time
combined_compare(r2_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
CPU times: user 1min 6s, sys: 1.51 s, total: 1min 7s
Wall time: 1min 4s


72.40808010519264

In [ ]:
%%time
combined_compare(w1_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
CPU times: user 53 s, sys: 1.38 s, total: 54.3 s
Wall time: 51.6 s


73.9834707326342

In [ ]:
%%time
combined_compare(w2_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
CPU times: user 44.5 s, sys: 1.22 s, total: 45.7 s
Wall time: 43.2 s


77.12152267900882

In [ ]:
%%time
combined_compare(w3_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
CPU times: user 52.6 s, sys: 1.29 s, total: 53.9 s
Wall time: 51.2 s


80.41409671364036

# VERSION 7.1 TEST

We identify and consider various relevant angles during the exercises and compare these with multiple trainer videos to obtain the best score out of the lot. This should also help us achieve angle invariance in each individual frames.

# VERSION 8.1 TEST: ANGLE INVARIANCE

- We rotate the coordinates using Euler's rotation matrix about `y-axis` for various angles and collect the coordinates for each frame.
- If we fix an angle $\theta$ to rotate, we obtain $\frac{2 \pi}{\theta}$ no. of coordinates for each key-point in each frame.
- In essence we obtain $\frac{2 \pi}{\theta}$ many videos to compare against the trainer video and we output the best score.

In [ ]:
# this causes anti-clockwise rotation w.r.t. the +ve y axis by an angle of theta
def Ry(theta):
  return np.matrix([[ np.cos(theta), 0,  np.sin(theta)],
                    [ 0            , 1,  0            ],
                    [-np.sin(theta), 0,  np.cos(theta)]])

In [ ]:
## to create a helper function to extract (n, 33, 3) dimensional array to store the coordinates of each landmark in each frame

## min_coordinates, max_coordinates = (n, 3), (n, 3)

def extract_arr(input_video_path):
    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            # initialize to store the key-points coordinates in the frame
            frame_coordinates = np.zeros((33,3))
            
            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z
                frame_coordinates[i][0], frame_coordinates[i][1], frame_coordinates[i][2] = x, y, z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z   
            
            # normalize frame_coordinates
            for i in range(33):
                # normalizing x coordinate
                frame_coordinates[i][0] = (frame_coordinates[i][0] - x_min)/(x_max - x_min)
                # normalizing y coordinate
                frame_coordinates[i][1] = (frame_coordinates[i][1] - y_min)/(y_max - y_min)
                # normalizing z coordinate
                frame_coordinates[i][2] = (frame_coordinates[i][2] - z_min)/(z_max - z_min)
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates)

In [ ]:
# extracts normalized coordinates in array of shape (n, 33, 3)
def extract_arr_rot(input_video_path, rot_angle=None):
    # if no rotation return the original coordinates array
    if rot_angle == None or rot_angle == 0:
        return extract_arr(input_video_path)

    # extract the rotation matrix
    rot_mat_y = Ry(rot_angle)

    coordinates = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass

            # initialize to store the key-points coordinates in the frame
            frame_coordinates = np.zeros((33,3))
            
            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z
                # rotate the coordinates by multiplying with rotation matrix
                C = rot_mat_y@np.array((x,y,z))
                x, y, z = C[0,0], C[0,1], C[0,2]

                frame_coordinates[i][0], frame_coordinates[i][1], frame_coordinates[i][2] = x, y, z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z   
            
            # normalize frame_coordinates
            for i in range(33):
                # normalizing x coordinate
                frame_coordinates[i][0] = (frame_coordinates[i][0] - x_min)/(x_max - x_min)
                # normalizing y coordinate
                frame_coordinates[i][1] = (frame_coordinates[i][1] - y_min)/(y_max - y_min)
                # normalizing z coordinate
                frame_coordinates[i][2] = (frame_coordinates[i][2] - z_min)/(z_max - z_min)
            
            coordinates.append(frame_coordinates)

        # Releasing the video capture device
        cap.release()

        return np.array(coordinates)    


In [ ]:
# reshape the (n, 33, 3) dimensional array to a (33, n, 3) dimensional array for extracting scores of each key-point

def my_reshape(input_arr):
    n_frames = input_arr.shape[0]
    n_landmarks = input_arr.shape[1]

    # new array of desired shape
    new_arr = np.zeros((n_landmarks, n_frames, 3))

    for f in range(n_frames):
        for i in range(n_landmarks):
            new_arr[i][f] = input_arr[f][i]
    
    return new_arr

In [ ]:
from dtaidistance.dtw_ndim import distance, distance_fast

## to compare 2 videos of dim (33, n, 3) using DTW (using strategy 2)

def compare_vid(vid1_arr, vid2_arr, senstivity = 1):
    n_landmarks = vid1_arr.shape[0]
    
    scores = np.zeros(n_landmarks)
    
    for i in range (n_landmarks):
        # normalize the coordinates using sensitivity; higher sensitivity => more lenient scoring, lower sensitivity => strict scoring
        vid1_arr[i] = vid1_arr[i]/(np.linalg.norm(vid1_arr[i]) * senstivity)
        vid2_arr[i] = vid2_arr[i]/(np.linalg.norm(vid2_arr[i]) * senstivity)
        
        # calculate distance
        d = distance(vid1_arr[i], vid2_arr[i])
        # we give a score out of 100
        d_score = 100 - (d*100)
        scores[i] = d_score
    
    return scores

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path, senstivity=1):
    vid1_arr = extract_arr(vid1_path)
    vid2_arr  = extract_arr(vid2_path)

    vid1_arr_new = my_reshape(vid1_arr)
    vid2_arr_new = my_reshape(vid2_arr)

    scores = compare_vid(vid1_arr_new, vid2_arr_new, senstivity)

    print("Scores List:\n", scores)
    print("Average Score:", scores.mean())

In [ ]:
# comparing videos by path: just including the pre-processing steps in the previous function
# we add an extra step of rotating the coordinates and collecting them for further comparison
# comparing (33, n, 3) dimensional vectors

def combined_compare_rot(vid1_path, vid2_path, senstivity=1, rot_angle=None):
    # list of scores
    scores = []
    
    vid2_arr  = extract_arr(vid2_path)
    vid2_arr_new = my_reshape(vid2_arr)
    
    # no rotation cases
    if rot_angle==None or rot_angle == 0:
        return combined_compare(vid1_path, vid2_path, senstivity)
    
    n_rotations = 2*np.pi//rot_angle

    for i in range(n_rotations):
        theta = rot_angle*i
        
        vid1_arr = extract_arr_rot(vid1_path, theta)
        vid1_arr_new = my_reshape(vid1_arr)

        score = compare_vid(vid1_arr_new, vid2_arr_new, senstivity).mean()
        scores.append(score)

    print(scores)
    return scores

# VERSION 8.2 TEST

In [ ]:
%%time